In [ ]:
%matplotlib notebook
#import matplotlib.pyplot as plt
#import numpy as np
import urllib.request
import gzip
import os

def load_covtype_dataset():
    '''Downloads the Cover Type dataset from UCI repository, returning a file handle'''
    CURRENT_DIR = os.getcwd()
    COVTYPE_FILENAME = 'covtype.data'
    COVTYPE_DATA_PATH = os.path.join(CURRENT_DIR, COVTYPE_FILENAME)
    COVTYPE_URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz'
    if os.path.isfile(COVTYPE_DATA_PATH):
        print('Using local cached copy in', COVTYPE_DATA_PATH)
    else:
        print('Dataset not found locally. Downloading in', COVTYPE_DATA_PATH)
        with urllib.request.urlopen(COVTYPE_URL) as response:
            with gzip.GzipFile(fileobj=response) as uncompressed, open(COVTYPE_DATA_PATH, 'wb') as out_file:
                file_header = uncompressed.read()
                out_file.write(file_header)
    return COVTYPE_DATA_PATH


In [ ]:
covtype_file = load_covtype_dataset()

In [ ]:
import pandas as pd
import numpy as np

df_covtype = pd.read_csv(covtype_file, header=None)

In [ ]:
df_covtype.describe()

In [ ]:
def features_renaming(df_covtype):
    '''Rename each column to meaningful labels'''
    # First step: rename the first 14 columns
    first_fourteen_old_feature_names = df_covtype.columns[np.arange(0,14)]
    first_fourteen_new_feature_names = ['Elevation', 'Aspect', 'Slope',
                                        'Horizontal_Distance_To_Hydrology',
                                        'Vertical_Distance_To_Hydrology',
                                        'Horizontal_Distance_To_Roadways',
                                        'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
                                        'Horizontal_Distance_To_Fire_Points',
                                        'Wilderness_Area_1', 'Wilderness_Area_2',
                                        'Wilderness_Area_3', 'Wilderness_Area_4']
    old_to_new_name_mapping = dict(zip(first_fourteen_old_feature_names, first_fourteen_new_feature_names))
    df_covtype.rename(columns=old_to_new_name_mapping, inplace=True)
    # Second step: rename the 40 soil type columns
    soil_type_old_feature_names = df_covtype.columns[np.arange(14,54)]
    soil_type_new_feature_names = ['Soil_Type_' + str(i) for i in np.arange(1,41)]
    old_to_new_name_mapping = dict(zip(soil_type_old_feature_names, soil_type_new_feature_names))
    df_covtype.rename(columns=old_to_new_name_mapping, inplace=True)
    # Last step: rename the last feature (cover type)
    df_covtype.rename(columns={54: 'Cover_Type'}, inplace=True)
    return df_covtype

In [ ]:
# Features names are numeric, let's rename each one of them
df_covtype = features_renaming(df_covtype)

In [ ]:
df_covtype.head()

In [ ]:
# Check the number of samples for each class value
df_covtype.Cover_Type.value_counts()

In [ ]:
# Let's copy the first 15120 rows onto a new df and
# perform some Exploratory Data Analysis (EDA)
train = df_covtype[:15120].copy()

In [ ]:
# Check if stratified sampling has already been done for the training set (it has)
train.Cover_Type.value_counts()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def elevation_multiple_covtype_distplot(elevations_df, covtype_mapping):
    '''Plot an Elevation's histogram for each Cover Type on the same figure'''
    # Color palette for distplot
    custom_palette = ['#F97EDB', '#49C0EA', '#B49EFC', '#CDAB40',
                      '#F69089', '#76BF3F', '#4CC9A6']
    # Set distplot background
    sns.set_style('darkgrid')
    # Custom bins range for evenly spaced hists
    bins = range(1800,4000,60)
    # Iterate through the dictionary to plot a histogram for each cover type
    for covtype_id, covtype_name in covtype_mapping.items():
        # covtype_id goes from 1 to 7
        # thus subtract 1 for indexing custom_palette
        palette_idx = covtype_id - 1        
        # Create a group for each Cover_Type and return a df satisfying the condition
        # on Cover_Type column
        by_one_covtype = elevations_df.groupby('Cover_Type') \
                                      .apply(lambda x: x[ x['Cover_Type'] == covtype_id ])
        # Plot one Elevation histogram for one group
        ax = sns.distplot(by_one_covtype.Elevation,
                          bins=bins,
                          color=custom_palette[palette_idx], label=covtype_name,
                          hist_kws=dict(alpha=0.8, edgecolor="none"),
                          kde=False)

    # Legend position to upper right
    plt.legend(loc='right', bbox_to_anchor=(1.2, 0.8), ncol=1)
    # Apply proper labeling to the axes
    ax.set(xlabel='Elevation (meters)', ylabel='Count')
    # Avoid cutting off the legend from the figure
    plt.tight_layout()
    # Show the figure (can be omitted in Jupyter Notebooks)
    plt.show()

In [ ]:
# Plot an Elevation's histogram for each Cover Type
# to check for possible class separation

# Slice by two columns: Elevation and Cover_Type
elevations = train.loc[:, ['Elevation', 'Cover_Type']]
# Dictionary for mapping each integer target label to its string value
covtype_label_name_dict = {1: 'Spruce/Fir',
                           2: 'Lodgepole Pine',
                           3: 'Ponderosa Pine',
                           4: 'Cottonwood/Willow',
                           5: 'Aspen',
                           6: 'Douglas-fir',
                           7: 'Krummholz'}
# It is clear that classes
# 4 (Willow), 5 (Aspen) and 7 (Krummholz) are easily separable
elevation_multiple_covtype_distplot(elevations, covtype_label_name_dict)

In [ ]:
# Don't need elevations anymore
print('Dereferencing elevations')
del elevations

In [ ]:
# Since the Elevation looks like a promising feature for Cover Type prediction
# let's check the correlation matrix to see which continuos feature
# depends on the Elevation
# Note that we could have used Chi-Squared Test to check for features
# dependency on Cover Type but it is out of scope for this project
from pandas.plotting import scatter_matrix

# The features we are interested in for the correlation matrix
features_to_check = ['Elevation', 'Aspect',
                     'Slope',
                     'Horizontal_Distance_To_Hydrology',
                     'Vertical_Distance_To_Hydrology',
                     'Horizontal_Distance_To_Roadways',
                     'Hillshade_9am', 'Hillshade_Noon',
                     'Hillshade_3pm',
                     'Horizontal_Distance_To_Fire_Points']
# Same features as above, just with some shorter names
# for visualization purposes
labels_to_plot = ['Elevation', 'Aspect',
                  'Slope', 'HD_Hydro',
                  'VD_Hydro', 'HD_Road',
                  'HS_9am', 'HS_Noon',
                  'HS_3pm', 'HD_Fire']
# Create a dictionary for mapping the labels like this
# {long_label: short_label}
shorter_labels = dict(zip(features_to_check, labels_to_plot))
# Make a copy of the training set because we want to rename
# some columns to fit them on the graph
corr_train = train[features_to_check].copy()
# Do the renaming feeding the mapping dictionary we created
corr_train.rename(columns=shorter_labels, inplace=True)
# Grab the AxesSubplots handle to modify labels padding and rotation
axes = scatter_matrix(corr_train, figsize=(12, 8))
n = len(corr_train.columns)
for x in range(n):
    for y in range(n):
        # for all the axes on the graph..
        ax = axes[x,y]
        # rotate the y-axis labels by 0 (horizontally)..
        ax.yaxis.label.set_rotation(0)
        # add some padding between the labels and their subgraph..
        ax.xaxis.labelpad = 0
        ax.yaxis.labelpad = 20
        # and hide axes value ranges
        ax.set_yticklabels([])
        ax.set_xticklabels([])

In [ ]:
# Let's write down the most correlated features:
# - 'HD_Hydro' and 'VD_Hydro'
# - 'HS_Noon' and 'HS_3pm'
# Plot the same scatter graph zooming on these
corr_train.plot(kind='scatter', x='HD_Hydro', y='VD_Hydro')

In [ ]:
# Looks like there are a lot of HS_3pm equal to zero.
# It could help to impute those with the median
corr_train.plot(kind='scatter', x='HS_Noon', y='HS_3pm')

In [ ]:
# Don't need corr_train df anymore
# Hopefully the garbage collector will clean it up
print('Dereferencing corr_train')
del corr_train

In [ ]:
# Count Hillshade_3pm zeros on the training set
(train.Hillshade_3pm == 0).astype(int).sum(axis=0)

In [ ]:
# Imputing Hillshade_3pm zeros with the median
from sklearn.impute import SimpleImputer

simp = SimpleImputer(missing_values=0, strategy='median')
# fit_transform requires X as a numpy array of shape [n_samples, n_features]
# thus the dataframe column is casted to a numpy array and reshaped
train.Hillshade_3pm = simp.fit_transform(train.Hillshade_3pm.values.reshape(-1,1))
# Count the zeros again to check the result
(train.Hillshade_3pm == 0).astype(int).sum(axis=0)

In [ ]:
# Drop the training set from df_covtype
df_covtype = df_covtype.iloc[15120:]
# Append the imputed training set to df_covtype
df_covtype = pd.concat([train, df_covtype])

In [ ]:
# Don't need train df anymore
del train

In [ ]:
# Let's compute the Euclidean distance to Hydrology
hv_distances_labels = ['Horizontal_Distance_To_Hydrology',
                       'Vertical_Distance_To_Hydrology']
hv_dist_hydro_arr = df_covtype[hv_distances_labels].values
# Perform the Euclidean distance with respect to the origin
# (i.e (0;0) where the water is located)
euc_distance_to_hydro = np.linalg.norm(hv_dist_hydro_arr, axis=1)
# Add the new feature 'Distance_To_Hydrology' to the training set
# just after the 'Slope' feature at index 2
# rounding each distance to four decimal places
df_covtype.insert(3,
                  'Distance_To_Hydrology',
                  np.around(euc_distance_to_hydro, decimals=4))
# Drop the horizontal and vertical distance
df_covtype.drop(columns=hv_distances_labels, inplace=True)

In [ ]:
# Check the result
df_covtype.head(2)

In [ ]:
# Perform one hot encoding via get_dummies,
# then drop the integer target label, leaving the one hot encoded labels only
one_hot_covtype = pd.get_dummies(df_covtype.Cover_Type, prefix='CovT')
df_covtype.drop(columns='Cover_Type', inplace=True)
df_covtype_ohe = df_covtype.join(one_hot_covtype)

In [ ]:
# Check the resulting shape
df_covtype_ohe.shape

In [ ]:
# Don't need df_covtype anymore
print('Dereferencing df_covtype')
del df_covtype

In [ ]:
# Downcast all features to reduce the overall dataframe dimension

# Get the columns header onto a list
df_covtype_ohe_headers = df_covtype_ohe.columns.values.tolist()
# There is no need to convert the last 7 elements, they are already uint8
# just drop them
df_covtype_ohe_headers = df_covtype_ohe_headers[:-7 or None]
# A list containing the new dtype for each column of interest
features_new_dtype_list = list()
# Iterate through the headers list to assign the new dtype
# for each column
for i in range(len(df_covtype_ohe_headers)):
    if i == 3:
        # Distance_To_Hydrology could fit in float16
        # but float32 works better with mean and std calculations
        features_new_dtype_list.append('float32')
    elif i < 9:
        # First nine features except Distance_To_Hydrology
        # can have int16
        features_new_dtype_list.append('int16')
    else:
        # the remaining ones are binary integers, so uint8
        # is enough
        features_new_dtype_list.append('uint8')
# A dictionary whose keys are the dataframe columns and values
# are the new dtype ( namely, {'Elevation' : 'int16', ...} )
features_new_dtype_mapping = dict(zip(df_covtype_ohe_headers,
                                      features_new_dtype_list))
# Perform the downcasting using the dictionary just created
df_covtype_ohe = df_covtype_ohe.astype(dtype=features_new_dtype_mapping)

In [ ]:
# Check the result
df_covtype_ohe.info()

In [ ]:
# Drop Aspect and Slope since working with angles is tricky
# when computing the mean
df_covtype_ohe.drop(labels=['Aspect', 'Slope'], axis=1, inplace=True)

In [ ]:
def train_test_split(df_covtype_ohe):
    '''
    Split the one hot encoded dataset onto training set and test set
    according to UCI's repository guidelines
    '''
    # First 15120 rows for the training set
    X_train = df_covtype_ohe[:15120].copy()
    # The last seven colums are the targets
    X_train, y_train = X_train.iloc[:, :51], X_train.iloc[:, 51:]
    # The remaining rows are for the test set
    X_test = df_covtype_ohe[15121:].copy()
    X_test, y_test = X_test.iloc[:, :51], X_test.iloc[:, 51:]
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split(df_covtype_ohe)

In [ ]:
# Check shapes consistency
print(f'X_train: {X_train.shape}, X_test: {X_test.shape}, ' \
      f'y_train: {y_train.shape}, y_test: {y_test.shape}')

In [ ]:
# Let's standardize the training set and test set.
# Note that we use the training set ONLY to calculate the mean and standard deviation
# then normalize the training set 
# and finally use the (training) mean and standard deviation to normalize the test set.
# This ensures no data leakage.

def train_test_normalize(X_train, X_test):
    '''
    Perform standardization on the training set and transforms the
    test set accordingly
    '''
    # The numerical columns we want to normalize
    numerical_columns = ['Elevation',
                         'Distance_To_Hydrology',
                         'Horizontal_Distance_To_Roadways',
                         'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
                         'Horizontal_Distance_To_Fire_Points']
    # Calculate the mean and standard deviation of the training set
    X_train_num_cols_mean = X_train[numerical_columns].mean()
    X_train_num_cols_std = X_train[numerical_columns].std()
    # Perform standardization over the numerical columns of the training set
    X_train_std = (X_train[numerical_columns] - X_train_num_cols_mean) / X_train_num_cols_std
    # Concatenate side-by-side the normalized training set and the one-hot encoded features
    # Note that we index X_train dataframe by the (set) difference of the overall features
    # minus the numerical ones
    ohe_features = X_train.columns.difference(other=numerical_columns, sort=False)
    X_train_std = pd.concat([X_train_std, X_train[ohe_features]], axis=1)
    # Perform standardization over the numerical columns of the test set, using the mean and std
    # of the training set as discussed earlier
    X_test_std = (X_test[numerical_columns] - X_train_num_cols_mean) / X_train_num_cols_std
    # Concatenate side-by-side the normalized test set and the one-hot encoded features
    X_test_std = pd.concat([X_test_std, X_test[ohe_features]], axis=1)
    return X_train_std, X_test_std

X_train_std, X_test_std = train_test_normalize(X_train, X_test)

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=0.1, random_state=1, solver='liblinear', multi_class='ovr')
lr.fit(X_train_std, y_train)

print('Logistic Regression, training accuracy: %.2f%%' % (100 * lr.score(X_train_std, y_train)))